##Assessing Science impact of PSV in LSST sensors

To assess the science impact of pixel size variation in LSST sensors, we will need to lay down sources with the following varying properties:
- focal plane position ( x in [50,3950]; y in [50,3950] with half-pixel step size
- ellipticity of source ( e in [0,1])
- orientation of source (theta in [0,180 degrees])

##Setting up ipcluster controller
After manually start cluster in "clusters" tab on home page

In [72]:
!bjobs

JOBID   USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
2224    mbaumer RUN   long       ki-ls09     hequ0012    *ontroller Jul 31 11:03
2233    mbaumer SSUSP long       ki-ls09     hequ0014    *engine[1] Jul 31 11:03
2233    mbaumer SSUSP long       ki-ls09     hequ0017    *engine[2] Jul 31 11:03
2233    mbaumer RUN   long       ki-ls09     fell0056    *engine[3] Jul 31 11:03
2233    mbaumer SSUSP long       ki-ls09     hequ0020    *engine[4] Jul 31 11:03
2233    mbaumer RUN   long       ki-ls09     hequ0022    *engine[5] Jul 31 11:03
2233    mbaumer SSUSP long       ki-ls09     hequ0027    *engine[6] Jul 31 11:03
2233    mbaumer RUN   long       ki-ls09     hequ0029    *engine[7] Jul 31 11:03
2233    mbaumer SSUSP long       ki-ls09     hequ0120    *engine[8] Jul 31 11:03
2233    mbaumer SSUSP long       ki-ls09     hequ0033    *engine[9] Jul 31 11:03
2233    mbaumer RUN   long       ki-ls09     fell0269    *ngine[10] Jul 31 11:03
2233    mbaumer SSUSP long   

In [46]:
a = !bjobs
len(a)

52

In [73]:
profile="lsf_parallel" # "wakari" will use ssh to remote nodes - change to "default" if necessary
from IPython.parallel import *
client = Client(profile=profile)
N_nodes = len(client.ids)
print 'Now controlling ' + str(N_nodes) + ' workers on LSF farm'
least_busy = client.load_balanced_view()
all_workers = client[:]
least_busy.block = False
all_workers.block   = False

def checkhostname():
    import socket
    return socket.gethostname()

hostnames = all_workers.apply(checkhostname)
print hostnames.get()

Now controlling 16 workers on LSF farm
['hequ0132', 'hequ0174', 'hequ0174', 'hequ0085', 'hequ0190', 'hequ0128', 'fell0302', 'fell0182', 'fell0182', 'hequ0139', 'hequ0022', 'fell0123', 'fell0123', 'fell0035', 'fell0269', 'fell0056']


In [8]:
import itertools
import numpy as np
evec = np.arange(0,.91,.1)
svec = np.arange(2,6.1,1)
tvec = np.arange(0,90.1,45)
param_vec = [item for item in itertools.product(evec,svec,tvec)]
local_str = 'I live on local'

@all_workers.parallel(block=True)
def runAna(param_vec):
    import numpy as np
    out = []
    for pars in param_vec:
        out.append(np.array(pars)**3)
    return out

#all_workers.scatter('param_vec',param_vec)

In [9]:
len(param_vec)

150

In [77]:
#param_vec = [[0,2,0],[.1,3,45]]

In [35]:
def doGridAna(params):
    import sys
    sys.path.append('/u/ki/mbaumer/random_pixel_size/weak_sauce/code')
    import pandas as pd
    import copy
    import numpy as np
    from weak_sauce.grid import MoveableGrid
    from weak_sauce.sources import Source
    from weak_sauce.movers import FixedIlluminationMover
    import galsim


    n_verts_x = 51

    saved_mg = MoveableGrid('../data/test100k_iter.pkl')
    #flat=np.load('../data/data_rel_flux_map_50x50.npy')
    flat=np.load('7th_order_LSST_50x50.npy')

    res_df_list = []
    ideal_df_list = []


    res_df = pd.DataFrame()
    ideal_df = pd.DataFrame()
    for xctr in np.arange(20,30):
        for yctr in np.arange(20,30):
            #params: e, sigma, theta
            obj = galsim.Gaussian(fwhm=params[1])
            obj = obj.shift(xctr,yctr)
            obj = obj.shear(e=params[0],beta=galsim.Angle(params[2],galsim.degrees))
            temp = copy.deepcopy(saved_mg)
            #source should be oversampled wrt pixels; hence the 5x
            stationary_source = Source(num_x=5*(n_verts_x-1)+1, min_x=0, max_x=50, \
                                       flux_func=obj)
            #stationary_source = Source(num_x=n_verts_x, flux_func=gauss, mu=(xctr,yctr),sigma=3) #non-oversampled
            illuminator = FixedIlluminationMover(stationary_source)

            ideal_grid = Source(num_x=n_verts_x)
            ideal_mg = MoveableGrid(ideal_grid,illuminator)
            ideal_mg.step()
            ideal_res = ideal_mg.evaluate_psf()
            ideal_sex_flux = ideal_mg.evaluate_sex()['FLUX_AUTO']
            ideal_res['FLUX_AUTO'] = ideal_sex_flux
            ideal_df = ideal_df.append(ideal_res)

            temp.source.fluxes -= 1
            #temp.source.psf_evaluator = Moment_Evaluator(num_iter_max = 1000)
            mg = MoveableGrid(temp.source, illuminator)
            mg.step()
            mg.source.fluxes /= flat
            fitted_res = mg.evaluate_psf()
            sex_flux = mg.evaluate_sex()['FLUX_AUTO']
            fitted_res['FLUX_AUTO'] = sex_flux
            res_df = res_df.append(fitted_res)
    res_df["inputE"] = params[0]
    res_df["inputS"] = params[1]
    res_df["inputTheta"] = params[2]
    ideal_df["inputE"] = params[0]
    ideal_df["inputS"] = params[1]
    ideal_df["inputTheta"] = params[2]
    res_df_list.append(res_df)
    ideal_df_list.append(ideal_df)
    return res_df_list, ideal_df_list

In [36]:
out = least_busy.map(doGridAna,param_vec,block=False,chunksize=1)

In [61]:
out.progress

150

In [62]:
if out.ready():
    if out.successful():
        print 'Everything Worked!'
    else:
        print 'Done, but something fucked up'
else:
    print 'Not ready yet...'

Everything Worked!


In [63]:
df = out.get()

In [81]:
a = df[0]
type(a[0][0])

pandas.core.frame.DataFrame

In [83]:
import pandas as pd
end_res = pd.DataFrame()
for i in np.arange(len(df)):
    current = df[i]
    temp = current[0][0]-current[1][0] #fitted-ideal
    temp['inputE'] = current[0][0]['inputE']
    temp['inputS'] = current[0][0]['inputS']
    temp['inputTheta'] = current[0][0]['inputTheta']
    end_res = end_res.append(temp)

In [118]:
end_res.columns

Index([u'Mx', u'Mxx', u'Mxy', u'My', u'Myy', u'a4', u'delta1', u'delta2',
       u'e0', u'e0prime', u'e1', u'e2', u'flux', u'fwhm', u'phi', u'w', u'w1',
       u'w2', u'wd1', u'wd2', u'whisker', u'x2', u'x2y', u'x3', u'xy', u'xy2',
       u'y2', u'y3', u'zeta1', u'zeta2', u'inputE', u'inputS', u'inputTheta'],
      dtype='object')

In [116]:
feat['flux'] = end_res['flux']/(.001*np.log(10)/2.5) #convert to mMags
feat[['Mx','My']] = 200*end_res[['Mx','My']] 
feat[['e1','e2']] = .2**2/.27**2*end_res[['e1','e2']]
feat[['inputE', 'inputS', 'inputTheta']] = end_res[['inputE', 'inputS', 'inputTheta']]

KeyboardInterrupt: 

In [119]:
print feat.shape
feat = feat.dropna()
print feat.shape
feat = feat[np.logical_and(feat['Mx'] > -1,feat['Mx'] < 1)]
print feat.shape
feat = feat[np.logical_and(feat['My'] > -1,feat['My'] < 1)]
print feat.shape
feat = feat[np.logical_and(feat['flux'] > -500,feat['My'] < 500)]
print feat.shape

(10644, 8)
(10644, 8)
(10644, 8)
(10644, 8)


##Plotting various biases (hold everything else constant)
- flux vs. size
- flux vs. ellipticity
- astrometry vs. size
- astrometry vs. ellipticity
- e1/e2 error vs. ellip
- e1/e2 error vs. size

In [123]:
grouped = feat.groupby(['inputE','inputS','inputTheta'])

In [125]:
stats = grouped.describe()

In [141]:
for s in svec
stats.loc[(0,2,0)]['Mx']['count']

100.0